# Steps for implementation

Reading CSV

Data Cleaning

Features

Applying HBOS to detect outliers from the data

In [1]:
#Importing Necessary Modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split     # import module for train test split
from pyod.models.hbos import HBOS
from pyod.utils.data import evaluate_print

In [2]:
def preprosess(df):

    #remove rows with any values that are not finite
    df = df[np.isfinite(df).all(1)]
    
    df=df.dropna( axis=0, how='any')
    df=df.replace(',,', np.nan, inplace=False)
    df=df.drop(columns=[' Fwd Header Length.1'], axis=1, inplace=False)

    df.replace("Infinity", 0, inplace=True)
    df['Flow Bytes/s'].replace("Infinity", 0,inplace=True)
    df[" Flow Packets/s"].replace("Infinity", 0, inplace=True)
    df[" Flow Packets/s"].replace(np.nan, 0, inplace=True)
    df['Flow Bytes/s'].replace(np.nan, 0,inplace=True)
    df["Bwd Avg Bulk Rate"].replace("Infinity", 0, inplace=True)
    df["Bwd Avg Bulk Rate"].replace(",,", 0, inplace=True)
    df["Bwd Avg Bulk Rate"].replace(np.nan, 0, inplace=True)
    df[" Bwd Avg Packets/Bulk"].replace("Infinity", 0, inplace=True)
    df[" Bwd Avg Packets/Bulk"].replace(",,", 0, inplace=True)
    df[" Bwd Avg Packets/Bulk"].replace(np.nan, 0, inplace=True)
    df[" Bwd Avg Bytes/Bulk"].replace("Infinity", 0, inplace=True)
    df[" Bwd Avg Bytes/Bulk"].replace(",,", 0, inplace=True)
    df[" Bwd Avg Bytes/Bulk"].replace(np.nan, 0, inplace=True)
    df[" Fwd Avg Bulk Rate"].replace("Infinity", 0, inplace=True)
    df[" Fwd Avg Bulk Rate"].replace(",,", 0, inplace=True)
    df[" Fwd Avg Bulk Rate"].replace(np.nan, 0, inplace=True)
    df[" Fwd Avg Packets/Bulk"].replace("Infinity", 0, inplace=True)
    df[" Fwd Avg Packets/Bulk"].replace(",,", 0, inplace=True)
    df[" Fwd Avg Packets/Bulk"].replace(np.nan, 0, inplace=True)
    df["Fwd Avg Bytes/Bulk"].replace("Infinity", 0, inplace=True)
    df["Fwd Avg Bytes/Bulk"].replace(",,", 0, inplace=True)
    df["Fwd Avg Bytes/Bulk"].replace(np.nan, 0, inplace=True)
    df[" CWE Flag Count"].replace("Infinity", 0, inplace=True)
    df[" CWE Flag Count"].replace(",,", 0, inplace=True)
    df[" CWE Flag Count"].replace(np.nan, 0, inplace=True)
    df[" Bwd URG Flags"].replace("Infinity", 0, inplace=True)
    df[" Bwd URG Flags"].replace(",,", 0, inplace=True)
    df[" Bwd URG Flags"].replace(np.nan, 0, inplace=True)
    df[" Bwd PSH Flags"].replace("Infinity", 0, inplace=True)
    df[" Bwd PSH Flags"].replace(",,", 0, inplace=True)
    df[" Bwd PSH Flags"].replace(np.nan, 0, inplace=True)
    df[" Fwd URG Flags"].replace("Infinity", 0, inplace=True)
    df[" Fwd URG Flags"].replace(",,", 0, inplace=True)
    df[" Fwd URG Flags"].replace(np.nan, 0, inplace=True)

    df["Flow Bytes/s"]=df["Flow Bytes/s"].astype("float64")
    df[' Flow Packets/s']=df[" Flow Packets/s"].astype("float64")
    df['Bwd Avg Bulk Rate']=df["Bwd Avg Bulk Rate"].astype("float64")
    df[' Bwd Avg Packets/Bulk']=df[" Bwd Avg Packets/Bulk"].astype("float64")
    df[' Bwd Avg Bytes/Bulk']=df[" Bwd Avg Bytes/Bulk"].astype("float64")
    df[' Fwd Avg Bulk Rate']=df[" Fwd Avg Bulk Rate"].astype("float64")
    df[' Fwd Avg Packets/Bulk']=df[" Fwd Avg Packets/Bulk"].astype("float64")
    df['Fwd Avg Bytes/Bulk']=df["Fwd Avg Bytes/Bulk"].astype("float64")
    df[' CWE Flag Count']=df[" CWE Flag Count"].astype("float64")
    df[' Bwd URG Flags']=df[" Bwd URG Flags"].astype("float64")
    df[' Bwd PSH Flags']=df[" Bwd PSH Flags"].astype("float64")
    df[' Fwd URG Flags']=df[" Fwd URG Flags"].astype("float64")
    pd.set_option('display.max_rows', df.shape[0])
    df.replace('Infinity',0.0, inplace=True)
    df.replace('NaN',0.0, inplace=True)


    # old_memory_usage = df.memory_usage().sum()
    #change the variable types for low memory usage
    #int64 to int32,,, float64 to float32
    integer = []
    f = []
    for i in df.columns[:-1]:
        if df[i].dtype == "int64": integer.append(i)
        else : f.append(i)

    df[integer] = df[integer].astype("int32")
    df[f] = df[f].astype("float32")

    # df = df.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)


    # print("============================")
    # print(df.info())
    return df


In [4]:
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.cblof import CBLOF
from pyod.models.loci import LOCI
from pyod.models.sos import SOS




df6 = pd.read_csv("./data/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
df6[" Label"] = df6[" Label"].replace(["Web Attack � Brute Force","Web Attack � XSS","Web Attack � Sql Injection"],"Web Attack")
# print(df6[" Label"].unique())
df6[' Label'].value_counts()

df6[' Label'] = df6[' Label'].replace(['BENIGN'], 0)
df6[' Label'] = df6[' Label'].replace(['Web Attack'], 1)

df = preprosess(df6)

features=[" Fwd Packet Length Max"," Flow IAT Std"," Fwd Packet Length Std" ,"Fwd IAT Total",' Flow Duration', " Fwd Packet Length Mean", " Total Length of Bwd Packets", "Total Length of Fwd Packets", " Flow IAT Mean", " Bwd Packet Length Mean",  " Flow IAT Max", " Bwd Packet Length Std", ' Total Fwd Packets', ' Total Backward Packets',' Label']

# print(df6.columns)
df = df[features].copy()

# print(df3[" Label"].unique())
print(df[' Label'].value_counts())
print(df.info())

X = df.drop([' Label'], axis=1)
y = df[' Label']
# X = df[df.columns[0:-1]]
# y = df[df.columns[-1]]

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2, random_state=10)

# X_train = X_train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(sampling_strategy=1) # Numerical value
# # rus = RandomUnderSampler(sampling_strategy="not minority") # String
# X_res, y_res = rus.fit_resample(X, y)

# ax = y_res.value_counts().plot.pie(autopct='%.2f')
# _ = ax.set_title("Under-sampling")

# # X_res.value_counts()
# print(y_res.value_counts())

X_train, X_test, y_train, y_test=train_test_split(X_train, y_train,test_size=0.2, random_state=10)

"""
# train kNN detector
clf_name = 'KNN'
# clf = KNN()
clf = KNN(contamination=0.01)
clf.fit(X_train)

# On Training Data:
# KNN ROC:0.1927, precision @ rank n:0.2119

# On Test Data:
# KNN ROC:0.2139, precision @ rank n:0.181
"""

"""
clf_name = 'LOF'
# clf = LOF(contamination=0.000001)
clf = LOF(contamination=0.1)

clf.fit(X_train)

# On Training Data:
# LOF ROC:0.355, precision @ rank n:0.4153
# On Test Data:
# LOF ROC:0.3764, precision @ rank n:0.4
"""

"""
clf_name = 'IForest'
clf = IForest(contamination=0.001)
clf.fit(X_train)

# On Training Data:
# IForest ROC:0.0989, precision @ rank n:0.1648
# On Test Data:
# IForest ROC:0.081, precision @ rank n:0.1238
"""

"""
clf_name = 'HBOS'
clf = HBOS(contamination=0.1)
clf.fit(X_train)

# HBOS ROC:0.4172, precision @ rank n:0.0036
# On Test Data:
# HBOS ROC:0.4117, precision @ rank n:0.0

"""

# # train LOCI detector
# clf_name = 'LOCI'
# clf = LOCI()
# clf.fit(X_train)

# print(X_train.shape)
# # train SOS detector
# clf_name = 'SOS'
# clf = SOS()
# clf.fit(X_train)



# clf_name = 'CBLOF'
# clf = CBLOF(contamination=0.1)
# clf.fit(X_train)

# # On Training Data:
# # CBLOF ROC:0.8898, precision @ rank n:0.0

# # On Test Data:
# # CBLOF ROC:0.8938, precision @ rank n:0.0

# # train CBLOF detector
# clf_name = 'CBLOF'
# clf = CBLOF(random_state=42)
# clf.fit(X_train)
# On Training Data:
# CBLOF ROC:0.8898, precision @ rank n:0.0

# On Test Data:
# CBLOF ROC:0.8938, precision @ rank n:0.0

"""
from pyod.models.loda import LODA

# train LOCI detector
clf_name = 'LODA'
clf = LODA()
clf.fit(X_train)

# On Training Data:
# LODA ROC:0.1066, precision @ rank n:0.0

# On Test Data:
# LODA ROC:0.1014, precision @ rank n:0.0

"""
"""
clf_name = 'HBOS'
clf = HBOS(contamination=0.09)
clf.fit(X_train)

# On Training Data:
# HBOS ROC:0.4172, precision @ rank n:0.0036

# On Test Data:
# HBOS ROC:0.4117, precision @ rank n:0.0
"""

"""

# Make predictions on test set
y_pred = clf.predict(X_test)

y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# get the prediction on the test data
y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(X_test)  # outlier scores

# print("\n", y_train)
# print("\n", y_train_scores)

# evaluate and print the results
print("\nOn Training Data:")
evaluate_print(clf_name, y_train, y_train_scores)
print("\nOn Test Data:")
evaluate_print(clf_name, y_test, y_test_scores)
"""


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix


# Initialize and train classifier model
clf = LogisticRegression(solver='lbfgs', max_iter=200).fit(X_train, y_train)
y_pred = clf.predict(X_test)

# Compare test set predictions with ground truth labels
print(accuracy_score(y_pred, y_test))
print(confusion_matrix(y_test, y_pred))

0    168051
1      2180
Name:  Label, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 170231 entries, 0 to 170365
Data columns (total 15 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0    Fwd Packet Length Max        170231 non-null  int32  
 1    Flow IAT Std                 170231 non-null  float32
 2    Fwd Packet Length Std        170231 non-null  float32
 3   Fwd IAT Total                 170231 non-null  int32  
 4    Flow Duration                170231 non-null  int32  
 5    Fwd Packet Length Mean       170231 non-null  float32
 6    Total Length of Bwd Packets  170231 non-null  int32  
 7   Total Length of Fwd Packets   170231 non-null  int32  
 8    Flow IAT Mean                170231 non-null  float32
 9    Bwd Packet Length Mean       170231 non-null  float32
 10   Flow IAT Max                 170231 non-null  int32  
 11   Bwd Packet Length Std        170231 non-null  float3

In [5]:
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

models = []
models.append(('KNN', KNeighborsClassifier()))
# models.append(('SVM', SVC(gamma='auto')))
models.append(('CART', DecisionTreeClassifier(max_depth=5)))
models.append(('RF', RandomForestClassifier(max_depth=5, n_estimators=5, max_features=3)))    
models.append(('ABoost', AdaBoostClassifier()))
models.append(('LR', LogisticRegression(solver='lbfgs', max_iter=200)))
models.append(('NB', GaussianNB()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('QDA', QuadraticDiscriminantAnalysis()))
models.append(('MLP', MLPClassifier()))

print('Model\tAcc\tPr\tRecall\tF1\tExecution')
      
for name, model in models:
    start_time = time.time()
    kfold = model_selection.KFold(n_splits=5, random_state=24, shuffle=True)    

    accuracy = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy').mean()
    precision = cross_val_score(model, X_train, y_train, cv=kfold, scoring='precision').mean()
    recall = cross_val_score(model, X_train, y_train, cv=kfold, scoring='recall').mean()
    f1_score = cross_val_score(model, X, y, cv=kfold, scoring='f1_weighted').mean()
    
    delta = time.time() - start_time
    print('{}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.2f} secs'.format(name, accuracy, precision, recall, f1_score, delta))

Model	Acc	Pr	Recall	F1	Execution
KNN	0.998	0.981	0.898	0.999	20.20 secs
CART	0.995	0.744	0.877	0.995	5.69 secs
RF	0.994	0.742	0.869	0.995	4.62 secs
ABoost	0.995	0.751	0.891	0.995	72.68 secs
LR	0.987	0.000	0.000	0.981	44.50 secs
NB	0.865	0.080	0.870	0.930	0.87 secs
LDA	0.987	0.470	0.045	0.982	4.03 secs
QDA	0.779	0.055	0.987	0.847	1.83 secs
MLP	0.986	0.083	0.016	0.979	276.78 secs


In [ ]:
from scikitplot.metrics import plot_confusion_matrix # For plotting confusion matrices
from sklearn.metrics import classification_report # Various metrics for model performance

model = KNeighborsClassifier(n_neighbors=5, weights='uniform',
                                    algorithm='auto', leaf_size=30,
                                    p=2, metric='minkowski',
                                    metric_params=None, n_jobs=None)

model.fit(X_train, y_train.ravel()) # 4)
pred_y = model.predict(X_test) # 5)
score = classification_report(y_test, pred_y) # 6)
print('Classification report: \n', score, '\n')

plot_confusion_matrix(y_test, pred_y, title='Confusion Matrix for {}'.format("KNeighborsClassifier"))

In [ ]:
from sklearn.neural_network import MLPClassifier # For Neural Network classifier

Neural_Net_model = MLPClassifier(hidden_layer_sizes=(10,), activation='relu',
                                    solver='adam', alpha=0.01, batch_size='auto',
                                    learning_rate='adaptive', learning_rate_init=0.1,
                                    max_iter=2)
    
# Fitting the model is synonymous to training the model. Need to call .ravel() to get array in correct format.
Neural_Net_model.fit(X_train, y_train.ravel())

# Using the model to predict the label/ classes, based upon X_test data only. This is the model's answers.
pred_y = Neural_Net_model.predict(X_test)

In [ ]:
# !pip install tensorflow

     |████████████████████████████████| 588.3 MB 15 kB/s  eta 0:00:016   |███▉                            | 71.3 MB 1.1 MB/s eta 0:08:05     |█████▍                          | 99.0 MB 505 kB/s eta 0:16:08     |█████▊                          | 104.9 MB 161 kB/s eta 0:49:46     |███████                         | 128.5 MB 603 kB/s eta 0:12:43     |██████████████▏                 | 260.9 MB 183 kB/s eta 0:29:42     |█████████████████████           | 387.6 MB 771 kB/s eta 0:04:21     |████████████████████████▎       | 447.1 MB 965 kB/s eta 0:02:27     |████████████████████████████▊   | 528.5 MB 1.0 MB/s eta 0:00:59
  Using cached tensorflow_estimator-2.11.0-py2.py3-none-any.whl (439 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
     |████████████████████████████████| 4.8 MB 1.3 MB/s eta 0:00:01
  Using cached keras-2.11.0-py2.py3-none-any.whl (1.7 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     |████████████████████████████████| 2.4 MB 292 kB/s eta 0:00:01
 

In [22]:
from itertools import cycle
import pickle
import sys
import time
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import LSTM, Bidirectional, BatchNormalization, Convolution1D, MaxPooling1D, Reshape
from tensorflow.keras.models import Sequential

%matplotlib inline

batch_size = 32
model = Sequential()
model.add(Convolution1D(64, kernel_size=32, padding="same",
          activation="relu", input_shape=(76, 1)))
model.add(MaxPooling1D(pool_size=(5)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(64, return_sequences=False)))
model.add(Reshape((128, 1), input_shape=(128, )))

model.add(MaxPooling1D(pool_size=(5)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(128, return_sequences=False)))

model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary(line_length=100)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = pd.DataFrame(columns=[1, 2, 3, 4, 5, 'Mean'],
                      index=['Accuracy', 'Precision', 'Recall', 'F1'])

ModuleNotFoundError: No module named 'tensorflow'